In [ ]:
''' Load audio, compute spectrogram and nmf.'''
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
import librosa
import IPython
import math
from scipy import stats
from sklearn.decomposition import NMF

def show(clip1, sr, n):
  duration = len(clip1)/sr
  print(len(clip1), max(clip1))
  f, t, Sxx = signal.spectrogram(clip1, sr, mode='magnitude',
                                 nperseg=n, nfft=n, noverlap=n-n//8, scaling='density')
  print('frequency resolution is: ', f[1])
  # scale by frequency, to get power per octave, or more radical scaling??
  # When scaled by f**3, we get a dramatic percussive signal.  This requires adding another NMF
  # component, that will emphasize the percussive element.  It's inconsistent, though, so probably
  # not worth the trouble.
  # Scaling by f**2, we get very nice definition of the bell spectra.
  Sxx = np.multiply(Sxx.T, np.arange(len(f))**2).T
  lp = librosa.logamplitude(Sxx)
    
  nmf = NMF(n_components=6)
  trans = nmf.fit_transform(Sxx.T)
  print('nmf shape: ', nmf.components_.shape)
  fig = plt.figure(figsize=(50,5), dpi=120)
  plt.plot(f, nmf.components_.T)
  plt.show()

  return nmf, trans


name = '/Users/gfr/Desktop/6 bell.wav'
#name = '/Users/gfr/Google Drive/Strikeometer_NewHope/audio/GSM/gsm-ringing-10/2013_02_25_Practice-part6.wav'
cambridge, sr = librosa.load(name)
print(sr)

nmf2048, trans2048 = show(cambridge, sr, 2048)
nmf512, trans512 = show(cambridge, sr, 512)
IPython.display.Audio(data=cambridge, rate=sr)


In [ ]:
''' Objective: use NMF with many components, but then group the components together into sequences.'''
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
import librosa
import IPython
import math
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF

def map(trans1, trans2):
    ''' Since we do two independent NMFs, we have to match up the resulting components.'''
    corr = np.corrcoef(trans1.T, trans2.T[:,2+4*np.arange(trans1.shape[0])])
    plt.figure(figsize=(4,4), dpi=200)
    plt.imshow(corr[:6,-6:], origin='lower') 
    plt.colorbar()
    plt.show()
    return np.argmax(corr[:6,-6:], axis=1)

def slow(sig, delta_t):
    # A half second gaussian for finding local energy.
    s = signal.gaussian(M=int(1.5/delta_t), std=.2/delta_t)
    s /= s.sum()
    return signal.convolve2d(sig, s.reshape(len(s),1), mode='same', boundary='symm')

def attack(sig):
    ''' For signals with rising edges of at least 4 steps, find the rising edges.'''
    fast = signal.gaussian(M=30, std=3)
    fast /= fast.sum()
    fast = fast[5:] - fast[:-5]
    return signal.convolve2d(sig, fast.reshape(len(fast),1), mode='same', boundary='symm')

def foo(trans1, dt1, trans2, dt2):
    t1 = np.arange(trans1.shape[0])*dt1
    t2 = np.arange(trans2.shape[0])*dt2
    
    s1 = slow(trans1, dt1)
    s1 /= np.max(s1, axis=0).T
    s2 = slow(trans2, dt2)
    s2 /= np.max(s2, axis=0).T
    
    a1 = attack(trans1)
    a1 /= np.max(a1, axis=0).T
    a2 = attack(trans2)
    a2 /= np.max(a2, axis=0).T
    
    a1s = np.stack((a1,a1,a1,a1), axis=1)
    a1s = np.reshape(a1s, [4*a1.shape[0], a1.shape[1]])
    f = signal.gaussian(M=14, std=2)
    f /= f.sum()
    a1sf = signal.convolve2d(a1s, f.reshape(len(f),1), mode='same', boundary='symm')
    a1sf = a1sf/np.max(a1s, axis=0).T
    fig = plt.figure(figsize=(100,12), dpi=120)
    plt.plot(t1[:len(t1)//5], s1[:len(t1)//5,] + np.arange(6).T)
    plt.plot(t2[:len(t2)//5], a2[:len(t2)//5,] + np.arange(6).T)
    plt.plot(t2[:a1s.shape[0]//5], a1s[:a1s.shape[0]//5] + np.arange(6).T)
    plt.plot(t2[:a1sf.shape[0]//5], a1sf[:a1sf.shape[0]//5] + np.arange(6).T)
    plt.show()

    print(a1sf.shape)
    ss = np.maximum(0, a1sf) + np.maximum(0, a2)[10:a1sf.shape[0]+10,:]
    ms = 2*np.maximum(0, a1sf) * np.maximum(0, a2)[10:a1sf.shape[0]+10,:]
    

    fig = plt.figure(figsize=(500,12), dpi=120)
    plt.plot(t2[:ss.shape[0]], ss + np.arange(6).T)
    plt.plot(t2[:ms.shape[0]], ms + np.arange(6).T)
    plt.plot(t2[:a1sf.shape[0]], np.maximum(0, a1sf/np.max(a1s, axis=0).T) + np.arange(6).T)
    plt.show()

sel = map(trans2048, trans512)

fast = signal.gaussian(M=30, std=3)
fast /= fast.sum()
fast = fast[5:] - fast[:-5]
fig = plt.figure(figsize=(8,4), dpi=120)
plt.plot(fast)
plt.show()


foo(trans2048[:,sel], 256/22100, trans512, 64/22100)


